## Libraries

In [1]:
import sys 
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\BARTScore")
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries") 

In [2]:
from custom_score.score import BERTScoreStaticSampleTest, BERTScoreDynamicSampleTest, BARTScoreDynamicSampleTest
from custom_score.utils import serialized_to_model
from bart_score import BARTScorer
import torch
import tensorflow_datasets as tfds
from datetime import datetime
import pandas as pd
import numpy as np

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Checkup and linkage

In [ ]:
torch.cuda.is_available()

## Datasets

### Billsum

In [3]:
billsum = tfds.load('huggingface:billsum')
billsum_test = tfds.as_dataframe(billsum["test"])
billsum_test = billsum_test.loc[:, ["text", "summary"]]
billsum_test.text = billsum_test.text.str.decode("utf-8")
billsum_test.summary = billsum_test.summary.str.decode("utf-8")
billsum_test.head(5)

,text,summary
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,Local Innovation and Coastal Protection Act of...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Gun Show Background Check Act of 2008 - Amends...
2,SECTION 1. SHORT TITLE.\n\n This Act may be...,Recycled Roads Act of 2003 - Directs the Secre...
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,Prosthetic and Custom Orthotic Parity Act of 2...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,"Investing in Neighborhood-focused, Vital, Evid..."


### Gigaword

In [9]:
gigaword_builder = tfds.builder("Gigaword")
gigaword_builder.download_and_prepare()
gigaword = gigaword_builder.as_dataset()

In [11]:
gigaword = tfds.as_dataframe(gigaword["test"])
gigaword = gigaword.rename(columns={"document":"text", "summary":"summary"})
gigaword['summary'] = gigaword['summary'].str.decode("utf-8").str.strip().str.replace("#", "")
gigaword['text'] = gigaword['text'].str.decode("utf-8").str.strip().str.replace("#", "")
gigaword.head(5)

,text,summary
0,the german government and red cross have decid...,"germany gives , dollars in aid for iran quake ..."
1,teenager michaela krajicek claimed her first g...,krajicek defeats safina in dutch final UNK pic...
2,after spending years in prison for a killing ...,"$ , for years wrongly held in prison"
3,the first probable human case of mad cow disea...,st case of probable mad cow disease listed pos...
4,at least people were killed as typhoon sepat ...,"storms lash chinese coast killing ; , evacuat..."


### Multinews

In [20]:
multinews_builder = tfds.builder("MultiNews")
multinews_builder.download_and_prepare()
multinews = multinews_builder.as_dataset()

In [21]:
multinews = tfds.as_dataframe(multinews["test"])
multinews = multinews.rename(columns={"document":"text", "summary":"summary"})
multinews['summary'] = multinews['summary'].str.decode("utf-8").str.replace("\xe2\x80\x93", "").str[2:].str.strip()
multinews['text'] = multinews['text'].str.decode("utf-8").str.strip()
multinews.head(5)

,text,summary
0,"Think back, everyone -- can you remember when ...",No matter how much you like Jason Bateman and ...
1,'Alien: Covenant': Film Review \n \n Michael ...,A spaceship arrives on a distant planet that l...
2,By REUTERS \n \n Photo: Reuters \n \n Charli...,France is on lockdown today after a satirical ...
3,These crawls are part of an effort to archive ...,Just when you thought the Republican president...
4,The estranged wife of acclaimed concert pianis...,"Sofya Tsygankova, the estranged wife of famed ..."


### Scientific Papers

In [ ]:
scipa = tfds.load('scientific_papers', split='train')

In [ ]:
scipa_builder = tfds.builder("scientific_papers")
scipa_builder.download_and_prepare()
scipa = scipa_builder.as_dataset()

In [ ]:
scipa = tfds.as_dataframe(scipa["test"])
scipa = scipa.rename(columns={"document":"text", "summary":"summary"})
#multinews['summary'] = multinews['summary'].str.decode("utf-8").str.replace("\xe2\x80\x93", "").str[2:].str.strip()
#multinews['text'] = multinews['text'].str.decode("utf-8").str.strip()
scipa.head(5)

## Benchmark

In [6]:
results = {}
lim = 10

### Billsum

In [ ]:
w2v = serialized_to_model(r'C:\Pro\Stages\A4 - DVRC\Work\Models\serialized_w2v.pkl')
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(billsum_test, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(billsum_test, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(billsum_test, limit=lim)

In [ ]:
billsum_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['billsum'] = billsum_results

In [7]:
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(billsum_test, limit=lim)

In [8]:
bart_scores

[0.34931090474128723,
 0.2687968909740448,
 0.305144339799881,
 0.2774801254272461,
 0.5581293702125549,
 0.426272451877594,
 0.41143250465393066,
 0.36529648303985596,
 0.49230146408081055,
 0.3428601622581482]

### Multinews

In [22]:
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(multinews, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(multinews, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(multinews, limit=lim)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaM

In [23]:
multinews_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['multinews'] = multinews_results

### Gigaword

In [24]:
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(gigaword, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(gigaword, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(gigaword, limit=lim)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaM

In [25]:
gigaword_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['gigaword'] = gigaword_results

### Scientific Papers

In [ ]:
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(scipa, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(scipa, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(scipa, limit=lim)

In [ ]:
scipa_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['scipa'] = scipa_results

## Evalutation

### Runtime Table

In [26]:
billsum_runtimes = [results["billsum"]["BERTScore"][1], results["billsum"]["Custom"][1], results["billsum"]["Bart"][1]]
multinews_runtimes = [results["multinews"]["BERTScore"][1], results["multinews"]["Custom"][1], results["multinews"]["Bart"][1]]
gigaword_runtimes = [results["gigaword"]["BERTScore"][1], results["gigaword"]["Custom"][1], results["gigaword"]["Bart"][1]]
#scipa_runtimes = [results["scipa"]["BERTScore"][1], results["scipa"]["Custom"][1], results["scipa"]["Bart"][1]]

runtimeDf = pd.DataFrame({"Billsum": billsum_runtimes, 
                          "Multinews": multinews_runtimes,
                          "Gigaword": gigaword_runtimes}, 
                         index=["Roberta-24-layers", "Word2Vec", "BART-large-CNN"])
runtimeDf

,Billsum,Multinews,Gigaword
Roberta-24-layers,54.698145,57.874716,53.497350
Word2Vec,42.865094,73.366389,0.070916
BART-large-CNN,2.323072,2.221653,0.459833


### Quality evaluation

#### Billsum

In [27]:
billsumQualityDf = pd.concat((pd.DataFrame(results["billsum"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["billsum"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["billsum"]["Bart"][0], columns=["Bart"])),
                              axis=1)
billsumQualityDf.head(5)

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart
0,0.646725,0.902619,0.753539,0.849191,0.781935,0.814176,0.998328
1,0.699015,0.941868,0.802470,0.818807,0.699549,0.754495,0.996797
2,0.652091,0.959232,0.776389,0.832273,0.697599,0.759008,0.996925
3,0.660414,0.971083,0.786169,0.813307,0.744471,0.777368,0.997079
4,0.500204,0.917080,0.647333,0.893293,0.700493,0.785231,0.995153


#### Multinews

In [28]:
multinewsQualityDf = pd.concat((pd.DataFrame(results["multinews"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["multinews"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["multinews"]["Bart"][0], columns=["Bart"])),
                              axis=1)
multinewsQualityDf.head(5)

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart
0,0.648935,0.854094,0.737513,0.810646,0.794206,0.802342,0.992231
1,0.656565,0.880320,0.752154,0.841052,0.826047,0.833482,0.995369
2,0.651669,0.863386,0.742735,0.863537,0.827867,0.845326,0.995048
3,0.617941,0.856395,0.717885,0.862562,0.805295,0.832945,0.990616
4,0.687182,0.878509,0.771155,0.914194,0.850664,0.881285,0.998107


#### Gigaword

In [29]:
gigawordQualityDf = pd.concat((pd.DataFrame(results["gigaword"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["gigaword"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["gigaword"]["Bart"][0], columns=["Bart"])),
                              axis=1)
gigawordQualityDf.head(5)

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart
0,0.520848,0.858084,0.648229,0.931568,0.861043,0.894918,0.996703
1,0.426644,0.616523,0.504302,0.878367,0.817973,0.847095,0.955153
2,0.397880,0.683416,0.502947,0.882140,0.839132,0.860099,0.956137
3,0.622144,0.939688,0.748635,0.947691,0.876814,0.910876,0.993289
4,0.445092,0.737083,0.555027,0.878134,0.835666,0.856374,0.983419


#### Scientific Papers

In [ ]:
scipaQualityDf = pd.concat((pd.DataFrame(results["scipa"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["scipa"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["scipa"]["Bart"][0], columns=["Bart"])),
                              axis=1)
scipaQualityDf.head(5)

#### Summary

In [30]:
meanGigawordQualityDf = gigawordQualityDf.mean(axis=0)
meanMultinewsQualityDf = multinewsQualityDf.mean(axis=0)
meanBillsumQualityDf = billsumQualityDf.mean(axis=0)
#meanScipaQualityDf = scipaQualityDf.mean(axis=0)

summaryDf = pd.DataFrame({"Gigaword":meanGigawordQualityDf,
                          "Billsum": meanBillsumQualityDf,
                          "Multinews": meanMultinewsQualityDf})
summaryDf

,Gigaword,Billsum,Multinews
W2V_P,0.429082,0.676538,0.675236
W2V_R,0.704253,0.930599,0.858380
W2V_F,0.529050,0.780857,0.752819
Bert_P,0.891676,0.842479,0.858570
Bert_R,0.835112,0.748533,0.819878
Bert_F,0.862067,0.791937,0.838675
Bart,0.968650,0.996443,0.994935


## Shuffled Data

### Shuffling datasets

In [64]:
shuffledGigaword = gigaword.copy()
shuffledGigaword["summary"] = shuffledGigaword["summary"].sample(frac=1).values

shuffledMultinews = multinews.copy()
shuffledMultinews["summary"] = shuffledMultinews["summary"].sample(frac=1).values

shuffledBillsum = billsum_test.copy()
shuffledBillsum["summary"] = shuffledBillsum["summary"].sample(frac=1).values

In [68]:
shuffledMultinews.head(3)

,text,summary
0,"Think back, everyone -- can you remember when ...",Serena Williams' loss to Naomi Osaka over the ...
1,'Alien: Covenant': Film Review \n \n Michael ...,When William McCormack went into a medical cen...
2,By REUTERS \n \n Photo: Reuters \n \n Charli...,A New York state appeals court ruled today tha...


In [69]:
shuffledBillsum.head(3)

,text,summary
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,Effective Terrorists Prosecution Act of 2006 -...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Establishes within the Department of Health an...
2,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Tax Relief Act of 2007 - Amends...


In [70]:
shuffledGigaword.head(3)

,text,summary
0,the german government and red cross have decid...,china marks gore visit with . billion dollar c...
1,teenager michaela krajicek claimed her first g...,china auto export slowdown in amid falling de...
2,after spending years in prison for a killing ...,a century of christian martyrdom


### Evaluating the forged datasets

In [71]:
results = {}
lim = 20

w2v = serialized_to_model(r'C:\Pro\Stages\A4 - DVRC\Work\Models\serialized_w2v.pkl')
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(shuffledBillsum, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(shuffledBillsum, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(shuffledBillsum, limit=lim)
billsum_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['billsum'] = billsum_results

bert_scores, bert_runtime = BERTScoreDynamicSampleTest(shuffledMultinews, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(shuffledMultinews, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(shuffledMultinews, limit=lim)
multinews_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['multinews'] = multinews_results

bert_scores, bert_runtime = BERTScoreDynamicSampleTest(shuffledGigaword, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(shuffledGigaword, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(shuffledGigaword, limit=lim)
gigaword_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['gigaword'] = gigaword_results

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaM

In [72]:
billsumQualityDf = pd.concat((pd.DataFrame(results["billsum"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["billsum"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["billsum"]["Bart"][0], columns=["Bart"])),
                              axis=1)
multinewsQualityDf = pd.concat((pd.DataFrame(results["multinews"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["multinews"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["multinews"]["Bart"][0], columns=["Bart"])),
                              axis=1)
gigawordQualityDf = pd.concat((pd.DataFrame(results["gigaword"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["gigaword"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["gigaword"]["Bart"][0], columns=["Bart"])),
                              axis=1)
meanGigawordQualityDf = gigawordQualityDf.mean(axis=0)
meanMultinewsQualityDf = multinewsQualityDf.mean(axis=0)
meanBillsumQualityDf = billsumQualityDf.mean(axis=0)

summaryDf = pd.DataFrame({"Gigaword":meanGigawordQualityDf,
                          "Billsum": meanBillsumQualityDf,
                          "Multinews": meanMultinewsQualityDf})
summaryDf

,Gigaword,Billsum,Multinews
W2V_P,0.249506,0.515896,0.555373
W2V_R,0.318810,0.634230,0.670574
W2V_F,0.276916,0.566491,0.605638
Bert_P,0.804658,0.794790,0.801455
Bert_R,0.790581,0.709324,0.773530
Bert_F,0.797279,0.748797,0.787198
Bart,0.921791,0.992752,0.991719
